# Setup



In [ ]:
%%capture
%pip install poetry
%pip install git+https://github.com/oughtinc/ergo.git@05b5eaf4735ea3c4f53802c6b3e71b03030d9634
%pip install xlrd

In [ ]:
%load_ext google.colab.data_table

In [ ]:
%%capture
import ergo
import numpy as np
import pandas as pd
import ssl
import warnings
from datetime import timedelta, date
from ergo.contrib.el_paso import texas_data, onlyasith, krismoore, brachbach
from ergo.contrib.utils import plot_question, question, rejection_sample, sample_from_ensemble, samplers, summarize_question_samples, daterange

In [ ]:
warnings.filterwarnings(action="ignore", category=FutureWarning)
warnings.filterwarnings(module="plotnine", action="ignore")
warnings.filterwarnings(module="jax", action="ignore")
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
metaculus = ergo.Metaculus(
    username="oughtpublic", 
    password="123456",
    api_domain = "pandemic"
)

# Retrieve external data and models



## Texas government cases data



In [ ]:
el_paso_cases = texas_data.get_el_paso_data()

el_paso_cases

## @KrisMoore's compiled data



Pulled from
[here](https://docs.google.com/spreadsheets/d/1eGF9xYmDmvAkr-dCmd-N4efHzPyYEfVl0YmL9zBvH9Q/edit#gid=1694267458)



In [ ]:
compiled_data = krismoore.get_krismoore_data()

compiled_data

## @onlyasith's cases model



Pulled from
[here](https://docs.google.com/spreadsheets/d/1L6pzFAEJ6MfnUwt-ea6tetKyvdi0YubnK_70SGm436c/edit#gid=1807978187)



In [ ]:
projected_cases = onlyasith.get_onlyasith_results()

projected_cases

## Shaman et al. cases model



Pulled from [here](https://github.com/shaman-lab/COVID-19Projection)
([paper](https://www.medrxiv.org/content/10.1101/2020.03.21.20040303v2))



In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
scenarios = ["nointerv", "60contact", "70contact", "80contact"]
cu_model_data = {}
for scenario in scenarios:
    df = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/Projection_{scenario}.csv", parse_dates=["Date"])
    df = df[df.county == "El Paso County TX"]
    scenarios = ["nointerv", "60contact", "70contact", "80contact"]
    scenario = scenarios[0]

    cases = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/Projection_{scenario}.csv", parse_dates=["Date"])
    cases = cases[cases.county == "El Paso County TX"]
    cases_left = cases[['county', 'fips', 'Date']]
    cases_left.loc[:,"var"] = 'cases'
    cases =  cases[['report_2.5', 'report_25', 'report_50', 'report_75', 'report_97.5']]
    cases.columns = cases.columns.map(lambda x: x.split('_')[1])
    cases = pd.concat([cases_left, cases], axis=1)


    multiple = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/bed_{scenario}.csv", parse_dates=["Date"])
    multiple = multiple[multiple.county == "El Paso County TX"]
    base = multiple[['county', 'fips', 'Date']]

    beds_left  = base.copy()
    beds_left.loc[:,"var"] = 'hosp'
    beds =  multiple[['hosp_need_2.5', 'hosp_need_25', 'hosp_need_50', 'hosp_need_75', 'hosp_need_97.5']]
    beds.columns = beds.columns.map(lambda x: x.split('_')[2])
    beds = pd.concat([beds_left, beds], axis=1)

    icu_left  = base.copy()
    icu_left.loc[:,"var"] = 'ICU'
    icu =  multiple[['ICU_need_2.5', 'ICU_need_25', 'ICU_need_50', 'ICU_need_75', 'ICU_need_97.5']]
    icu.columns = icu.columns.map(lambda x: x.split('_')[2])
    icu = pd.concat([icu_left, icu], axis=1)

    vent_left  = base.copy()
    vent_left.loc[:,"var"] = 'vent'
    vent =  multiple[['vent_need_2.5', 'vent_need_25', 'vent_need_50', 'vent_need_75', 'vent_need_97.5']]
    vent.columns = vent.columns.map(lambda x: x.split('_')[2])
    vent = pd.concat([vent_left, vent], axis=1)

    death_left  = base.copy()
    death_left.loc[:,"var"] = 'death'
    death =  multiple[['death_2.5', 'death_25', 'death_50', 'death_75', 'death_97.5']]
    death.columns = death.columns.map(lambda x: x.split('_')[1])
    death = pd.concat([death_left, death], axis=1)

    df = pd.concat([cases, beds, icu, vent, death])
    df["Date"] = df["Date"].apply(lambda x: x.date())
    #df.set_index("Date", inplace = True)
    cu_model_data[scenario] = df

@ergo.mem
def cu_model_scenario():
    """Which of the model scenarios are we in?"""
    return ergo.random_choice([s for s in cu_model_data.keys()])

@ergo.mem
def cu_model_quantile():
    """Where in the distribution of model outputs are we for this model run?
    Want to be consistent across time, so we sample it once per model run"""
    return ergo.uniform()

In [ ]:
scenarios = ["nointerv", "60contact", "70contact", "80contact"]
scenario = scenarios[0]

cases = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/Projection_{scenario}.csv", parse_dates=["Date"])
cases = cases[cases.county == "El Paso County TX"]
cases_left = cases[['county', 'fips', 'Date']]
cases_left.loc[:,"id"] = 'cases'
cases =  cases[['report_2.5', 'report_25', 'report_50', 'report_75', 'report_97.5']]
cases.columns = cases.columns.map(lambda x: x.split('_')[1])
cases = pd.concat([cases_left, cases], axis=1)


multiple = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/bed_{scenario}.csv", parse_dates=["Date"])
multiple = multiple[multiple.county == "El Paso County TX"]
base = multiple[['county', 'fips', 'Date']]

beds_left  = base.copy()
beds_left.loc[:,"id"] = 'hosp'
beds =  multiple[['hosp_need_2.5', 'hosp_need_25', 'hosp_need_50', 'hosp_need_75', 'hosp_need_97.5']]
beds.columns = beds.columns.map(lambda x: x.split('_')[2])
beds = pd.concat([beds_left, beds], axis=1)

icu_left  = base.copy()
icu_left.loc[:,"id"] = 'ICU'
icu =  multiple[['ICU_need_2.5', 'ICU_need_25', 'ICU_need_50', 'ICU_need_75', 'ICU_need_97.5']]
icu.columns = icu.columns.map(lambda x: x.split('_')[2])
icu = pd.concat([icu_left, icu], axis=1)

vent_left  = base.copy()
vent_left.loc[:,"id"] = 'vent'
vent =  multiple[['vent_need_2.5', 'vent_need_25', 'vent_need_50', 'vent_need_75', 'vent_need_97.5']]
vent.columns = vent.columns.map(lambda x: x.split('_')[2])
vent = pd.concat([vent_left, vent], axis=1)

death_left  = base.copy()
death_left.loc[:,"id"] = 'death'
death =  multiple[['death_2.5', 'death_25', 'death_50', 'death_75', 'death_97.5']]
death.columns = death.columns.map(lambda x: x.split('_')[1])
death = pd.concat([death_left, death], axis=1)

df = pd.concat([cases, beds, icu, vent, death])
df["Date"] = df["Date"].apply(lambda x: x.date())
df.set_index("Date", inplace = True)
#cu_model_data[scenario] = df


#df["Date"] = df["Date"].apply(lambda x: x.date())
#df.set_index("Date", inplace = True)
#cu_model_data[scenario] = df

## @brachbach model: cases -> hospitalized



In [ ]:
get_daily_hospital_confirmed = brachbach.get_daily_hospital_confirmed

## Access historical data



In [ ]:
def get_historical_data(date: date, column_name):
    """
    Look up the value of a parameter on a given date
    in the historical data we've loaded.

    Return the value or raise a KeyError if we don't have it.
    """

    # prefer Texas government data over @KrisMoore compiled data
    try: 
        value = el_paso_cases.loc[date, column_name]
        if np.isnan(value):
            raise KeyError(f"value for {column_name} in el_paso_cases is NaN")
    except KeyError:
        value = compiled_data.loc[date, column_name]
        if np.isnan(value):
            raise KeyError(f"value for {column_name} in compiled_data is NaN")
    return value

# Model components



In this notebook, we model some aspects of how the COVID-19 pandemic
will play out in El Paso. Our goal is to answer the questions in the
[El
Paso Metaculus series](https://pandemic.metaculus.com/questions/?search=cat:internal--el-paso).

In this section, we model some key variables that we'll use to answer
the Metaculus questions in the next section. We sometimes ensemble
multiple models to try to get a better estimate of a variable.



In [ ]:
START_DATE = date(2020, 4, 1)

## Daily COVID Infections



### Shaman Model



In [ ]:
def daily_infections_cu_model(date: date) -> int:
    """
    Predict the number of reported (new) Covid-19 infections on [date]
    using the Columbia model
    """
    scenario = cu_model_scenario()
    quantile = cu_model_quantile()

    # Extract quantiles of the model distribution
    xs = np.array([0.025, 0.25, 0.5, 0.75, 0.975])
    ys = cu_model_data[scenario].loc[(cu_model_data[scenario]["Date"] == date) & (cu_model_data[scenario]["var"] == "cases") , ["2.5", "25",  "50",  "75",  "97.5"]].values[0]

    # Linearly interpolate
    return int(round(np.interp(quantile, xs, ys)))

scenario = cu_model_scenario()
quantile = cu_model_quantile()

### @onlyasith model+



In [ ]:
@ergo.mem
def daily_infections_base_model(date: date) -> int:
    """
    What is the number of reported (new) Covid-19 infections on [date]?
    """
    try:
        # Look up projections from @onlyasith's model
        cases = projected_cases.loc[date, "New cases"]
        if np.isnan(cases):
            raise KeyError

        # Add some (fairly arbitrary) uncertainty around this point estimate
        if cases == 0:
          return cases
        cases_estimate = ergo.lognormal_from_interval(cases * 0.8, cases * 1.2)
        return int(np.clip(cases_estimate, cases * 0.5, cases * 2).round())
    except KeyError:
        # We're beyond the time range for data and model
        return 0

### Sample from ensemble



In [ ]:
@ergo.mem
def daily_infections(date: date) -> int:
    """
    What is the number of reported (new) Covid-19 infections on [date]?
    """
    try:
        return get_historical_data(date, "New cases")
    except KeyError:  # if we don't have historical data, use our ensemble of models
        return sample_from_ensemble([daily_infections_cu_model, daily_infections_base_model], {"date": date}, [.8, .2], fallback=True, default=0)

## Daily COVID infections: mean, sma, peak



In [ ]:
@ergo.mem
def mean_infections(start_date: date, end_date: date):
    """
    What is the average number of reported new infections for this range of 
    dates? (Including start date, excluding end date)
    """
    days = daterange(start_date, end_date)
    return np.mean([daily_infections(day) for day in days])

In [ ]:
@ergo.mem
def sma_infections(date: date):
    """
    The simple moving average of infections for a date.

    Defined in https://pandemic.metaculus.com/questions/4128:

    'The 2-day SMA is defined as the unweighted average (arithmetic mean)
    over the current day and the previous day.'
    """
    return mean_infections(date - timedelta(1), date + timedelta(1))

@ergo.mem
def peak_compatible_with_historical_data(peak_date):
    """
    Check whether it's possible that some date in the past
    was the peak date of new COVID infections in El Paso,
    given the historical data we have about COVID infections
    """
    if not peak_date in el_paso_cases.index:
        return True
    for comparison_date in daterange(START_DATE, peak_date + timedelta(11)):
        if comparison_date not in el_paso_cases.index:
            continue
        if sma_infections(comparison_date) > sma_infections(peak_date):
            return False
        if sma_infections(comparison_date) == sma_infections(peak_date) and comparison_date > peak_date:
            return False
    return True


@ergo.mem
def peak_infection_date_community():
    """
    The community assigns probability to some dates in the past
    that we already know were not the peak.
    So instead of sampling from the full community distribution,
    sample from the portion of the community distribution
    that is plausibly correct.
    """    
    peak_date = rejection_sample(
        peak_infection_date.question.sample_community, 
        peak_compatible_with_historical_data)
    return peak_date

## Patients with COVID in the hospital



In [ ]:
# Build @brachbach model
hospital_confirmed_from_daily_infected_model = get_daily_hospital_confirmed(compiled_data, daily_infections)

@ergo.mem
def hospital_confirmed_for_date(date: date) -> int:
    """
    The total number of lab-confirmed COVID-19 patients in El Paso County in
    the hospital on this date
    """
    try:
        return get_historical_data(date, "In hospital confirmed")
    except KeyError:
        try:
            # Get point estimate from @brachbach's regression model
            cases = hospital_confirmed_from_daily_infected_model(date)

            # Add some (fairly arbitrary) uncertainty around this point estimate
            if cases == 0:
              return cases
            cases_estimate = ergo.lognormal_from_interval(cases * 0.8, cases * 1.2)
            return np.clip(cases_estimate, cases * 0.5, cases * 2)
        except KeyError:
            return 0

## Proportion ICU admissions requiring ventilation



In [ ]:
@ergo.mem
def frac_icu_ventilation():
    """
    Proportion of ICU admissions requiring ventilation

    Approach (PabloStafforini et al): 
    https://pandemic.metaculus.com/questions/4154/#comment-28155

    TODO: 
    - Improve how we use case data
    - Add qualitative adjustments
    """
    ventilation_pseudocounts = 25 + 17 + 0.05 * 1150 + 0.1 * 132
    icu_pseudocounts = 100 + 36 + 0.05 * 1300 + 0.1 * 196
    return ergo.beta_from_hits(ventilation_pseudocounts, icu_pseudocounts)

# El Paso questions



In [ ]:
@question(metaculus, 4128, community_weight=0.95, community_fn=peak_infection_date_community, start_date=START_DATE)
def peak_infection_date() -> date:
    """
    When will El Paso County, Texas, experience its first peak number of COVID
    infections?

    From https://pandemic.metaculus.com/questions/4128:
    'This question resolves as the date for which
    the 2-day simple moving average(SMA) of the number of reported new infections
    is strictly greater than the 2-day SMA over the subsequent 10 days.'
    """
    end_date = date(2020, 7, 1)
    for today in daterange(START_DATE, end_date):
        sma_today = sma_infections(today)
        future_smas = [sma_infections(today + timedelta(i)) for i in range(1,11)]
        if sma_today > max(future_smas):
            return today
            return end_date

plot_question(peak_infection_date, start_date=START_DATE)

In [ ]:
@question(metaculus, 4137, community_weight=0.5)
def peak_infections():
    """
    How many new infections will be reported in El Paso on the day on which
    the number of new reported infections peaks?
    """
    peak = peak_infection_date()
    return daily_infections(peak)
plot_question(peak_infections)

In [ ]:
@question(metaculus, 4152, community_weight=0.5)
def mean_infections_peak345():
    """
    What will the average number of reported daily infections be in El Paso,
    over the 3rd, 4th and 5th days after the first "peak"?
    """
    peak = peak_infection_date()
    return mean_infections(peak + timedelta(3), peak + timedelta(6))
plot_question(mean_infections_peak345)

In [ ]:
@question(metaculus, 4170, community_weight=0.8)
def mean_infections_peak678():
    """
    What will the average number of reported daily infections be in El Paso,
    over the 6th, 7th and 8th days after the first "peak"?  
    """
    peak = peak_infection_date()
    return mean_infections(peak + timedelta(6), peak + timedelta(9))
plot_question(mean_infections_peak678)

In [ ]:
@question(metaculus, 4155, community_weight=0.7)
def frac_patients_icu():
    """
    What portion of in-hospital cases in El Paso County will require admission
    to the ICU?

    Following @katifish's approach:
    https://pandemic.metaculus.com/questions/4155/#comment-28054

    TODO: Add others from katifish comment
    """
    alpha = 0.1 # Rescaling counts becase we're more uncertain than implied by counts
    return ergo.random_choice([
      ergo.beta_from_hits(alpha * 121, alpha * 508),
      ergo.beta_from_hits(alpha * 181, alpha * 507),
    ])
plot_question(frac_patients_icu)

In [ ]:
@question(metaculus, 4154, community_weight=0.3)
def frac_patients_invasive():
    """
    What portion of in-hospital patients with Covid-19 in El Paso County will
    require invasive ventilation?

    Following @PabloStafforini's indirect estimation approach:
    https://pandemic.metaculus.com/questions/4154/#comment-28155

    TODO:
    - Combine with direct estimate
      direct_estimate = ergo.beta_from_hits(0.1 * 130, 0.1 * 393)
    """
    return frac_patients_icu() * frac_icu_ventilation()
plot_question(frac_patients_invasive)

In [ ]:
@ergo.mem
def peak_hospitalized_date():
    """
    What will be the date when there are the max number of COVID patients in the hospital
    within 15 days before or after the date of the first peak in confirmed cases?
    """
    infection_peak_date = peak_infection_date()
    days = list(daterange(infection_peak_date - timedelta(15), infection_peak_date + timedelta(16)))

    hospitalization_peak_date = days[0]
    hospitalized_peak = 0

    for day in days:
        hospitalized_for_day = hospital_confirmed_for_date(day)

        # if there are 2 different dates
        # with the same peak number of hospitalized patients,
        # return the first date:
        # https://pandemic.metaculus.com/questions/4204#comment-30023
        if hospitalized_for_day > hospitalized_peak:
            hospitalization_peak_date = day
            hospitalized_peak = hospitalized_for_day

    return hospitalization_peak_date

@question(metaculus, 4153, community_weight=0.3)
def max_30d_hospital_confirmed_for_peak():
    """
    What will the maximum number of in-hospital lab-confirmed COVID-19 
    patients in El Paso County, in the 30-day period during which the "peak"
    occurs?
    """
    return hospital_confirmed_for_date(peak_hospitalized_date())

plot_question(max_30d_hospital_confirmed_for_peak, bw=0.01)

In [ ]:
@question(metaculus, 4204)
def peak_icu_patients():
    """
    How many patients with Covid-19 in El Paso County will be in the
    ICU on the day when the number of hospital admissions of cases peak? 
    """
    try:
        return get_historical_data(peak_hospitalized_date(), "in_icu")
    except KeyError:
        # If we don't have historical data,
        # use the community prediction...
        if ergo.flip(0.8):
            return peak_icu_patients.question.sample_community()
        else:
            # ...or use @Tamay's approach:
            # https://pandemic.metaculus.com/questions/4204/
            peak_hospitalized_patients = max_30d_hospital_confirmed_for_peak()
            return peak_hospitalized_patients * frac_patients_icu()
plot_question(peak_icu_patients, bw=0.1)

In [ ]:
@question(metaculus, 4201)
def peak_invasive_ventilation():
    """
    How many patients with Covid-19 in El Paso County will be on invasive 
    ventilation on the day when the number of hospital admissions of cases 
    peak?
    """
    try:
        return get_historical_data(peak_hospitalized_date(), "on_ventilator")
    except KeyError:
        # If we don't have historical data,
        # use the community prediction...
        if ergo.flip(0.8):
            return peak_invasive_ventilation.question.sample_community()
        else:
            # ...or use @Tamay's approach:
            # https://pandemic.metaculus.com/questions/4201/#comment-28004
            return frac_icu_ventilation() * peak_icu_patients()


plot_question(peak_invasive_ventilation, bw=0.1)

# Generate predictions for all questions



In [ ]:
def model():
    for sampler in samplers.values():
        sampler()

samples = ergo.run(model, num_samples=2000)

summarize_question_samples(samples)

# Compare predictions to community



This takes a while since we're fitting a mixture of logistic
distributions to our samples before visualizing (and submitting) them.
These may look a little different from the plots for the questions shown
above, because we've taken more samples from the distribution and we're
fitting logistic distributions so we can submit them to metaculus



In [ ]:
submissions = {}
for sampler in samplers.values():
    q = sampler.question

    q_samples = samples[sampler.__name__]

    if q.id == 4128: # Date question: Need to convert back to date from days (https://github.com/oughtinc/ergo/issues/144)
        q_samples = np.array([START_DATE + timedelta(s) for s in q_samples])

    if q.id in [4201, 4204, 4137, 4152, 4170, 4153]:
      # Clip extreme values for questions that we had issues fitting
      (sample_min, sample_max) = np.quantile(q_samples, [0.08, 0.94])
      q_samples = q_samples[(q_samples >= sample_min) & (q_samples <= sample_max)]

    submission = q.get_submission_from_samples(q_samples)
    submissions[q] = submission

    # the graph for this question will be too zoomed out unless we cut off more of the graph
    if q.id == 4153:
      q.show_prediction(q_samples, plot_samples=False, plot_fitted=True, show_community=True, percent_kept=0.7)
    else:
      q.show_prediction(q_samples, plot_samples=False, plot_fitted=True, show_community=True, percent_kept=0.9)

In [ ]:
# Should we submit this to Metaculus? If so, uncomment the following lines:
# for q, submission in submissions.items():  
#     print(q.submit(submission))